In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import LFPy
import numpy as np
import h5py
import scipy.signal as ss
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
from matplotlib.collections import PolyCollection, PatchCollection
from plotting import annotate_subplot

In [ ]:
plt.rcParams.update({
    'axes.xmargin': 0.01,
    'axes.ymargin': 0.01,
    'font.size': 14,
    'legend.fontsize': 12,
    'axes.titlesize': 14,
})

In [ ]:
# Dataset
OUTPUTPATH = os.path.join('output', 'adb947bfb931a5a8d09ad078a6d256b0')
OUTPUTPATH_APPROX = os.path.join('output', 'f0a4032067f1fef454d5369276fb1cc9')


In [ ]:
gs = GridSpec(7, 2)
fig = plt.figure(figsize=(16, 16))

np.random.seed(1234)

ax = fig.add_subplot(gs[:5, 0])
ax.axis('off')

cell_xz = [
    [-500, -100],
    [500, -40],
    [-300, 80],
    [300, 60],
]

for i, (x, z) in enumerate(cell_xz):
    morphology='BallAndSticks_short.hoc'
    cellParameters = dict(
        morphology=morphology,
        templatefile='BallAndSticksTemplate.hoc',
        templatename='BallAndSticksTemplate',
        templateargs=None,
        delete_sections=True,
    )
    cell = LFPy.TemplateCell(**cellParameters)
    cell.set_pos(x, 0, z)

    ax.text(x+10, z-20, r'$\mathrm{unit}\:{%i}$' % (i+1), ha='left', va='top')

    # show morphology
    zips = []
    for x, z in cell.get_idx_polygons(projection=('x', 'z')):
        zips.append(list(zip(x, z)))
    polycol = PolyCollection(zips,
                             edgecolors='k',
                             facecolors='C{}'.format(i))
    ax.add_collection(polycol)


# connections
for i, (x_post, z_post) in enumerate(cell_xz):
    z_syn = np.random.rand(len(cell_xz)) * 600 - 100 + z_post
    # ax.plot([x_post] * (z_syn.size - 1), z_syn[np.arange(z_syn.size) != i], 'ro')
    for j, (x_pre, z_pre) in enumerate(cell_xz):
        if i != j:
            # draw line from presyn soma to synapse
            ax.plot([x_pre, x_post + (-np.sign(x_post-x_pre)*15)], 
                    [z_pre,  z_syn[j]], 'o-', color='C{}'.format(j))

# LFP data
f = h5py.File(os.path.join(OUTPUTPATH, 'RecExtElectrode.h5'), 'r')
data = f['data']['imem'][::2, -1601:]
data = (data.T - data.mean(axis=-1)).T
t = np.linspace(200, 500, data.shape[1])
for i, x in enumerate(data):
    ax.plot(t, x*1e2 - i * 50 + 900, 'gray', lw=1)

ax.text(350, 925, r'$V_\mathrm{e}(\mathbf{R}, t)$', color='gray', fontsize=16, ha='center', va='bottom')

    
# probe
el = [list(zip([-40, -40, 0, 40, 40, -40], 
               [500, -50, -100, -50, 500, 500]))]
elpoly = PolyCollection(el,
                        edgecolors='gray',
                        facecolors='w')
ax.add_collection(elpoly)
ax.plot(np.zeros(data.shape[0]), np.linspace(-50, 450, data.shape[0]), 'o', color='gray')

# spike train sequences
for i in range(len(cell_xz)):
    ax.step(np.arange(100) * 2 - 300, 
             np.random.binomial(1, 0.025, size=100)*40 - 375 - i * 50, 
             color='C{}'.format(i), where='mid')
    ax.text(-300, - 375 - i * 50, r'$s_%i(t)$' % (i+1), ha='right')

ax.text(-200, -400 -i * 50, 'spikes', ha='center', va='top')


    
# database
patches = []
patches.append(mpatches.Ellipse((200, -500), 200, 50))
patches.append(mpatches.Rectangle((100, -500), 200, 100))
patches.append(mpatches.Ellipse((200, -400), 200, 50))
patches.append(mpatches.Ellipse((200, -390), 200, 50))
patches.append(mpatches.Ellipse((200, -380), 200, 50))

# arrows
patches += [mpatches.Arrow(x=-50, y=-450, dx=100, dy=0, width=100)]
patches += [mpatches.Arrow(x=-200, y=-200, dx=0, dy=-100, width=100)]

db = PatchCollection(patches, edgecolors='w', facecolors='k')
ax.add_collection(db)

# ax.add_patch(mpatches.Arrow(0, 500, 150, 200, 100, fc='gray'))
style = "Simple, tail_width=15, head_width=50, head_length=30"
kw = dict(arrowstyle=style, color="gray")
ax.add_collection(PatchCollection([mpatches.FancyArrowPatch((0, 500), (175, 750), connectionstyle="arc3,rad=-0.2", **kw)], 
                                  edgecolors='gray', facecolors='gray'))



# touch up
ax.add_patch(mpatches.Ellipse((200, -500), 195, 45, fc='k', ec='none', lw=0))

ax.axis(ax.axis('equal'))

annotate_subplot(ax, ncols=2, nrows=0.5, letter='A', linear_offset=0)



np.random.seed(1234)

ax = fig.add_subplot(gs[:5, 1], sharex=ax)
ax.axis('off')
# ax.grid('on')

cell_xz = [
    [-500, -100],
    [500, -40],
    [-300, 80],
    [300, 60],
]

for i, (x, z) in enumerate(cell_xz):
    morphology='BallAndSticks_short.hoc'
    cellParameters = dict(
        morphology=morphology,
        templatefile='BallAndSticksTemplate.hoc',
        templatename='BallAndSticksTemplate',
        templateargs=None,
        delete_sections=True,
    )
    cell = LFPy.TemplateCell(**cellParameters)
    cell.set_pos(x, 0, z)

    ax.text(x+10, z-20, r'$\mathrm{unit}\:{%i}$' % (i+1), fontsize=12, ha='left', va='top')

    # show morphology
    zips = []
    for x, z in cell.get_idx_polygons(projection=('x', 'z')):
        zips.append(list(zip(x, z)))
    polycol = PolyCollection(zips,
                             edgecolors='k',
                             facecolors='C{}'.format(i))
    ax.add_collection(polycol)
    

# connections
for i, (x_post, z_post) in enumerate(cell_xz):
    z_syn = np.random.rand(len(cell_xz)) * 600 - 100 + z_post
    for j, (x_pre, z_pre) in enumerate(cell_xz):
        if i != j:
            # draw line from presyn soma to synapse
            ax.plot([x_pre, x_post + (-np.sign(x_post-x_pre)*15)], 
                    [z_pre,  z_syn[j]], '--', color='C{}'.format(j), alpha=0.5)
            ax.plot([x_pre, x_post + (-np.sign(x_post-x_pre)*15)], 
                    [z_pre,  z_syn[j]], 'o', color='C{}'.format(j))
            
# LFP data
f = h5py.File(os.path.join(OUTPUTPATH, 'RecExtElectrode.h5'), 'r')
data = f['data']['imem'][::2, -1601:]
data = (data.T - data.mean(axis=-1)).T
t = np.linspace(200, 500, data.shape[1])
for i, x in enumerate(data):
    ax.plot(t, x*1e2 - i * 50 + 900, 'k', lw=1)

ax.text(350, 925, r'$V_\mathrm{e}(\mathbf{R}, t)$', fontsize=16, ha='center', va='bottom')

    
# probe
el = [list(zip([-40, -40, 0, 40, 40, -40], 
               [500, -50, -100, -50, 500, 500]))]
elpoly = PolyCollection(el,
                        edgecolors='k',
                        facecolors='w')
ax.add_collection(elpoly)
ax.plot(np.zeros(data.shape[0]), np.linspace(-50, 450, data.shape[0]), 'ko')

# spike train sequences
for i in range(len(cell_xz)):
    ax.step(np.arange(100) * 2 + 100, 
            np.random.binomial(1, 0.025, size=100)*40 - 375 - i * 50, 
            color='C{}'.format(i), where='mid')
    ax.text(300, - 375 - i * 50, r'$(s_%i \ast \delta_{\Delta uv})(t)$' % (i+1), ha='left')

ax.text(200, -400 -i * 50, 'synapse\nactivation\ntimes', ha='center',va='top')
    
# database
patches = []
patches.append(mpatches.Ellipse((-200, -400), 200, 50))
patches.append(mpatches.Rectangle((-300, -500), 200, 100))
patches.append(mpatches.Ellipse((-200, -400), 200, 50))
patches.append(mpatches.Ellipse((-200, -390), 200, 50))
patches.append(mpatches.Ellipse((-200, -380), 200, 50))

# arrows
patches += [mpatches.Arrow(x=-50, y=-450, dx=100, dy=0, width=100)]
patches += [mpatches.Arrow(x=200, y=-300, dx=0, dy=100, width=100)]
# patches += [mpatches.Arrow(0, 500, 150, 200, 100)]
style = "Simple, tail_width=15, head_width=50, head_length=30"
kw = dict(arrowstyle=style, color="C0")
ax.add_collection(PatchCollection([mpatches.FancyArrowPatch((0, 500), (175, 750), connectionstyle="arc3,rad=-0.2", **kw)], 
                                  edgecolors='k', facecolors='k'))


db = PatchCollection(patches, edgecolors='w', facecolors='k')
ax.add_collection(db)

# touch up
ax.add_patch(mpatches.Ellipse((-200, -500), 195, 45, fc='k', ec='none', lw=0))

ax.axis(ax.axis('equal'))

annotate_subplot(ax, ncols=2, nrows=0.5, letter='B', linear_offset=0)



np.random.seed(1234)

ax = fig.add_subplot(gs[5:7, 0], sharex=ax)
ax.axis('off')

cell_xz = [
    [-500, -100],
    [500, -40],
    [-300, 80],
    [300, 60],
]

for i, (x, z) in enumerate(cell_xz):
    morphology='BallAndSticks_E.hoc'
    cellParameters = dict(
        morphology=morphology,
        templatefile='BallAndSticksTemplate.hoc',
        templatename='BallAndSticksTemplate',
        templateargs=None,
        delete_sections=True,
    )
    cell = LFPy.TemplateCell(**cellParameters)
    cell.set_pos(x, 0, z)

    ax.text(x+10, z-20, r'$\mathrm{unit}\:{%i}$' % (i+1), ha='left', va='top')
    
    # show morphology
    zips = []
    for x, z in cell.get_idx_polygons(projection=('x', 'z')):
        zips.append(list(zip(x, z)))
        break
    polycol = PolyCollection(zips,
                             edgecolors='k',
                             facecolors='C{}'.format(i))
    ax.add_collection(polycol)


# connections
for i, (x_post, z_post) in enumerate(cell_xz):
    np.random.rand(len(cell_xz)) * 600 - 100 + z_post
    z_syn = [z_post] * len(cell_xz) # np.random.rand(len(cell_xz)) * 1200 - 200 + z_post
    for j, (x_pre, z_pre) in enumerate(cell_xz):
        if i != j:
            # draw line from presyn soma to synapse
            ax.plot([x_pre, x_post + (-np.sign(x_post-x_pre)*15)], 
                    [z_pre,  z_syn[j]], 'o-', color='C{}'.format(j))
            continue


# spike train sequences
for i in range(len(cell_xz)):
    ax.step(np.arange(100) * 2 - 300, 
            np.random.binomial(1, 0.025, size=100)*40 - 325 - i * 50, 
            color='C{}'.format(i), where='mid')
    ax.text(-300, - 325 - i * 50, r'$s_%i(t)$' % (i+1), ha='right')

ax.text(-200, -350 -i * 50, 'spikes', ha='center', va='top')

# database
patches = []
patches.append(mpatches.Ellipse((200, -350), 200, 50))
patches.append(mpatches.Rectangle((100, -450), 200, 100))
patches.append(mpatches.Ellipse((200, -350), 200, 50))
patches.append(mpatches.Ellipse((200, -340), 200, 50))
patches.append(mpatches.Ellipse((200, -330), 200, 50))

# arrows
patches += [mpatches.Arrow(-50, -400, 100, 0, 100)]
patches += [mpatches.Arrow(-200, -150, 0, -100, 100)]

db = PatchCollection(patches, edgecolors='w', facecolors='k')
ax.add_collection(db)

# touch up
ax.add_patch(mpatches.Ellipse((200, -450), 195, 45, fc='k', ec='none', lw=0))

ax.axis(ax.axis('equal'))

annotate_subplot(ax, ncols=2, nrows=0.5, letter='C', linear_offset=0)


fig.savefig('Figure_4.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
import json
import hashlib
from parameters import ParameterSpace
from example_network_parameters import (networkParameters, population_names,
                                        population_sizes)

def get_H_YX():
    PS2 = ParameterSpace('PS2.txt')

    # TRANSIENT = 200
    dt = networkParameters['dt']
    tau = 25  # time lag relative to spike for kernel predictions

    for k, pset in enumerate(PS2.iter_inner()):
        # sorted json dictionary
        js = json.dumps(pset, sort_keys=True).encode()
        md5 = hashlib.md5(js).hexdigest()

        for i, (X, t, N_X) in enumerate(zip(population_names,
                                            [pset['t_E'], pset['t_I']],
                                            population_sizes)):
            inds = (np.arange(-tau // dt, tau // dt + 1)
                    + t // dt).astype(int)
            for j, Y in enumerate(population_names):
                if (X == 'I') & (Y == 'E'):
                    for h, (unit, fname) in enumerate(
                            zip(['mV', ],
                                ['RecExtElectrode.h5', ])):
                        title = (
                            r'$\langle H_\mathrm{%s %s}(\mathbf{r}, \tau) \rangle$'
                            % (Y, X)
                            )

                        with h5py.File(os.path.join('output', md5, fname), 'r') as f:
                            H_YX = f['data'][Y][:, inds] / N_X
                            H_YX = (H_YX.T - H_YX[:, 0]).T
                else:
                    continue
    return H_YX

In [ ]:
gs = GridSpec(6, 2)
fig = plt.figure(figsize=(16, 10))

np.random.seed(1234)


np.random.seed(1234)

ax = fig.add_subplot(gs[:, 0], sharex=ax)
ax.axis('off')

for i, (x, z) in enumerate(cell_xz):
    morphology='BallAndSticks_short.hoc'
    cellParameters = dict(
        morphology=morphology,
        templatefile='BallAndSticksTemplate.hoc',
        templatename='BallAndSticksTemplate',
        templateargs=None,
        delete_sections=True,
    )
    cell = LFPy.TemplateCell(**cellParameters)
    cell.set_pos(x, 0, z)

    ax.text(x+10, z-20, r'$\mathrm{unit}\:{%i}$' % (i+1), fontsize=12, ha='left', va='top')

    # show morphology
    zips = []
    for x, z in cell.get_idx_polygons(projection=('x', 'z')):
        zips.append(list(zip(x, z)))
    polycol = PolyCollection(zips,
                             edgecolors='k',
                             facecolors='C{}'.format(i))
    ax.add_collection(polycol)
    

# connections
for i, (x_post, z_post) in enumerate(cell_xz):
    z_syn = np.random.rand(len(cell_xz)) * 600 - 100 + z_post
    for j, (x_pre, z_pre) in enumerate(cell_xz):
        if i != j:
            # draw line from presyn soma to synapse
            ax.plot([x_pre, x_post + (-np.sign(x_post-x_pre)*15)], 
                    [z_pre,  z_syn[j]], '--', color='C{}'.format(j), alpha=0.5)
            ax.plot([x_pre, x_post + (-np.sign(x_post-x_pre)*15)], 
                    [z_pre,  z_syn[j]], 'o', color='C{}'.format(j))
            
# kernel
data = get_H_YX()[::2]
t = np.linspace(200, 500, data.shape[1])
for i, x in enumerate(data):
    ax.plot(t, x*1e3 - i * 50 + 900, 'k', lw=1)

ax.text(350, 925, r'$H(\mathbf{R}, \tau)$', fontsize=16, ha='center', va='bottom')

    
# probe
el = [list(zip([-40, -40, 0, 40, 40, -40], 
               [500, -50, -100, -50, 500, 500]))]
elpoly = PolyCollection(el,
                        edgecolors='k',
                        facecolors='w')
ax.add_collection(elpoly)
ax.plot(np.zeros(data.shape[0]), np.linspace(-50, 450, data.shape[0]), 'ko')

# spike train sequences
st = np.zeros(100)
st[50] = 1
for i in range(len(cell_xz)):
    ax.step(np.arange(100) * 2 + 100, 
            st*40 - 375 - i * 50,
            color='C{}'.format(i), where='mid')
    ax.text(300, - 375 - i * 50, r'$(s_%i \ast \delta_{\Delta uv})(t)$' % (i+1), size=12, ha='left')

ax.text(200, -400 -i * 50, 'synapse\nactivation\ntimes', size=12, ha='center',va='top')
    
# database
patches = []
patches.append(mpatches.Ellipse((-200, -400), 200, 50))
patches.append(mpatches.Rectangle((-300, -500), 200, 100))
patches.append(mpatches.Ellipse((-200, -400), 200, 50))
patches.append(mpatches.Ellipse((-200, -390), 200, 50))
patches.append(mpatches.Ellipse((-200, -380), 200, 50))

# arrows
patches += [mpatches.Arrow(x=-50, y=-450, dx=100, dy=0, width=100)]
patches += [mpatches.Arrow(x=200, y=-300, dx=0, dy=100, width=100)]

db = PatchCollection(patches, edgecolors='w', facecolors='k')
ax.add_collection(db)


style = "Simple, tail_width=15, head_width=50, head_length=30"
kw = dict(arrowstyle=style, color="C0")
ax.add_collection(PatchCollection([mpatches.FancyArrowPatch((0, 500), (175, 750), connectionstyle="arc3,rad=-0.2", **kw)], 
                                  edgecolors='k', facecolors='k'))

# touch up
ax.add_patch(mpatches.Ellipse((-200, -500), 195, 45, fc='k', ec='none', lw=0))

ax.axis(ax.axis('equal'))

annotate_subplot(ax, ncols=2, nrows=0.5, letter='A', linear_offset=0)


###############################
ax = fig.add_subplot(gs[:3, 1], sharex=ax)
ax.axis('off')

# spike train sequences
st = np.zeros(100)
st[0] = 1
for i in range(len(cell_xz)):
    ax.step(np.arange(100) * 3 - 150, 
            # st*40 + 25 - i * 50,
            np.random.binomial(1, 0.025, size=100)*40 + 75 - i * 50, 
            color='C{}'.format(i), where='mid')
    ax.text(150, 75 - i * 50, r'$s_%i(t)$' % (i+1), ha='left')

ax.text(0, 50 -i * 50, 'spikes', size=12, ha='center',va='top')

# binning
ax.plot([0, 0], [200, -90], 'k', lw=1)
ax.plot([10, 10], [200, -90], 'k', lw=1)
ax.text(5, 205, '$\Delta t$', ha='center', va='bottom')
ax.plot(-5, 200, 'k>')
ax.plot(15, 200, 'k<')


# database
patches = []
patches.append(mpatches.Ellipse((-400, 50), 200, 50))
patches.append(mpatches.Rectangle((-500, -50), 200, 100))
patches.append(mpatches.Ellipse((-400, 50), 200, 50))
patches.append(mpatches.Ellipse((-400, 60), 200, 50))
patches.append(mpatches.Ellipse((-400, 70), 200, 50))

# arrows
patches += [mpatches.Arrow(x=-275, y=0, dx=100, dy=0, width=100)]
patches += [mpatches.Arrow(x=0, y=-200, dx=0, dy=-100, width=100)]

db = PatchCollection(patches, edgecolors='w', facecolors='k')
ax.add_collection(db)

# touch up
ax.add_patch(mpatches.Ellipse((-400, -50), 195, 45, fc='k', ec='none', lw=0))

# spike rate
with h5py.File(os.path.join(OUTPUTPATH, 'RecExtElectrode.h5'), 'r') as f:
    bins = np.arange(f['data']['imem'].shape[1] // 16)[-1601:].astype(float)[::10]
with h5py.File(os.path.join(OUTPUTPATH, 'spikes.h5'), 'r') as f:
    times = np.array([], dtype=float)
    for X in ['E', 'I']:
        times = np.concatenate((times, np.concatenate(f[X]['times'][()])))
hist, _ = np.histogram(times, bins=bins)
bins -= bins[0]
bins /= bins[-1]
bins *= 300
bins -= 150
hist = hist - hist.mean()
# smoothen rate profiles
w = ss.windows.gaussian(11, 3)
w /= w.sum()
hist = np.convolve(hist, w, 'same')
hist *= 2  # scaling
ax.plot(bins[:-1], hist - 400, 'k')

ax.text(0, -500, r'spike rate $\nu(t)$', size=12, ha='center',va='top')
ax.axis(ax.axis('equal'))

annotate_subplot(ax, ncols=2, nrows=0.5, letter='B', linear_offset=0)


ax = fig.add_subplot(gs[3:, 1], sharex=ax)
ax.axis('off')

# spike rate
with h5py.File(os.path.join(OUTPUTPATH, 'RecExtElectrode.h5'), 'r') as f:
    bins = np.arange(f['data']['imem'].shape[1] // 16)[-1601:].astype(float)[::10]
bins -= bins[0]
bins /= bins[-1]
bins *= 300
bins -= 500
ax.plot(bins[:-1], hist - 100, 'k')

ax.text(-350, 50, r'$\nu(t)$', fontsize=16, ha='center', va='center')
ax.text(-150, 50, r'$\ast$', fontsize=16, ha='center', va='center')


# kernel
data = get_H_YX()[::2]
t = np.linspace(-100, 200, data.shape[1])
for i, x in enumerate(data):
    ax.plot(t, x*1e3 - i * 50 + 0, 'k', lw=1)

ax.text(50, 50, r'$H(\mathbf{R}, \tau)$', fontsize=16, ha='center', va='center')
ax.text(250, 50, r'$=$', fontsize=16, ha='center', va='center')


# LFP data (approximated)
f = h5py.File(os.path.join(OUTPUTPATH_APPROX, 'RecExtElectrode.h5'), 'r')
data = f['data']['imem'][::2, -1601:]
data = (data.T - data.mean(axis=-1)).T
t = np.linspace(300, 600, data.shape[1])
for i, x in enumerate(data):
    ax.plot(t, x*1e2 - i * 50 + 0, 'k', lw=1)

ax.text(450, 50, r'$\breve{V}_\mathrm{e}(\mathbf{R}, t)$', fontsize=16, ha='center', va='center')
ax.axis(ax.axis('equal'))

annotate_subplot(ax, ncols=2, nrows=0.5, letter='C', linear_offset=0)

fig.savefig('Figure_7.pdf', bbox_inches='tight', pad_inches=0)